# Lightweight Fine-Tuning Project

This project explores the application of Parameter-Efficient Fine-Tuning (PEFT) techniques, focusing on LoRA (Low-Rank Adaptation), to fine-tune the GPT-2 model. The primary aim is to enhance the model's understanding and prediction of sentiment within the financial news domain, specifically leveraging data sourced from Twitter.

## Loading and Evaluating a Foundation Model

In [1]:
from datasets import load_dataset
dataset = load_dataset("zeroshot/twitter-financial-news-sentiment")

/root/anaconda3/envs/projects/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_name = "ProsusAI/finbert"
id2label = {
    0: "Bearish", 
   1: "Bullish", 
    2: "Neutral"
}  
label2id=dict(zip(id2label.values(),id2label.keys()))



model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3,id2label=id2label, label2id=label2id)
tokenizer = AutoTokenizer.from_pretrained(model_name)


In [3]:
def preprocess_dataset(dataset):
    tokenized_dataset=tokenizer(dataset['text'],truncation=True,return_tensors="pt",padding=True)
    return tokenized_dataset
dataset=dataset.map(preprocess_dataset,batched=True)

Map: 100%|██████████| 2388/2388 [00:00<00:00, 11243.86 examples/s]


In [4]:
for split in ["train",'validation']:
    dataset[split]=dataset[split].rename_column('label','labels')

dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 9543
    })
    validation: Dataset({
        features: ['text', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 2388
    })
})

In [5]:
from transformers import TrainingArguments, Trainer
import numpy as np

def compute_metrics(pred_eval):
    predictions,target = pred_eval
    pred_labels = np.argmax(predictions,axis=1)
    return {"accuracy": (pred_labels==target).mean()}

training_args=TrainingArguments(output_dir="./results_before_finetuning",
    per_device_eval_batch_size=8)
trainer=Trainer(
model=model,
args=training_args,
eval_dataset=dataset['validation'],
    compute_metrics=compute_metrics
)

In [6]:
trainer.evaluate()

{'eval_loss': 1.5094716548919678,
 'eval_model_preparation_time': 0.0033,
 'eval_accuracy': 0.5251256281407035,
 'eval_runtime': 10.3207,
 'eval_samples_per_second': 231.381,
 'eval_steps_per_second': 28.971}

## Performing Parameter-Efficient Fine-Tuning

In [11]:
from peft import LoraConfig, TaskType

lora_config = LoraConfig(task_type=TaskType.SEQ_CLS)

In [12]:
from peft import get_peft_model
lora_model = get_peft_model(model,lora_config,)

In [13]:
from transformers import TrainingArguments, Trainer, DataCollatorWithPadding
import numpy as np


training_args = TrainingArguments(output_dir="results",
                                       learning_rate=0.002, 
                                       per_device_train_batch_size=9, 
                                       per_device_eval_batch_size=9, 
                                       evaluation_strategy="epoch",
                                      save_strategy="epoch",
                                       num_train_epochs=2,
                                 load_best_model_at_end=True)
trainer=Trainer(model=lora_model,
                args=training_args, 
                train_dataset= dataset['train'], 
                eval_dataset=dataset['validation'], 
                compute_metrics=compute_metrics, 
                
                data_collator=DataCollatorWithPadding(tokenizer), 
                tokenizer=tokenizer)


/root/anaconda3/envs/projects/lib/python3.12/site-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [15]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.488700,0.428146,0.836265
2,0.308600,0.379581,0.872697


TrainOutput(global_step=2122, training_loss=0.42176432344398895, metrics={'train_runtime': 99.033, 'train_samples_per_second': 192.724, 'train_steps_per_second': 21.427, 'total_flos': 852784885553760.0, 'train_loss': 0.42176432344398895, 'epoch': 2.0})

In [16]:
save_path="saved_models"
lora_model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

('saved_models/tokenizer_config.json',
 'saved_models/special_tokens_map.json',
 'saved_models/vocab.txt',
 'saved_models/added_tokens.json',
 'saved_models/tokenizer.json')

## Performing Inference with a PEFT Model

In [17]:
from peft import AutoPeftModelForSequenceClassification
lora_model_loaded=AutoPeftModelForSequenceClassification.from_pretrained(save_path)
tokenizer=AutoTokenizer.from_pretrained(save_path)

In [18]:
training_args=TrainingArguments(output_dir="./results_after_finetuning",
    per_device_eval_batch_size=8)
trainer=Trainer(model=lora_model_loaded,
                args=training_args, 
                eval_dataset=dataset['validation'], 
                compute_metrics=compute_metrics, )
trainer.evaluate()

{'eval_loss': 0.3795812726020813,
 'eval_model_preparation_time': 0.0019,
 'eval_accuracy': 0.8726968174204355,
 'eval_runtime': 4.3638,
 'eval_samples_per_second': 547.231,
 'eval_steps_per_second': 68.518}